In [ ]:
! pip install transformers
! pip install datasets
!pip install torchmetrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score


In [ ]:
from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time


import torch
from sklearn.metrics import accuracy_score

In [ ]:

import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation

Mounted at /content/drive
/content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation


In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:

df_unsupervised = pd.read_json('/content/drive/MyDrive/新建文件夹/ERAI_Dataset_unsupervised.json')

In [ ]:
df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
# df2 = pd.read_json('ERAI_Dataset_pairwise_test.json')

from sklearn.utils import shuffle
df = shuffle(df)
# df2 = shuffle(df2)


df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))
# df2['post1']=df2['post1'].apply(lambda x:x.replace('\n',''))
# df2['post2']=df2['post2'].apply(lambda x:x.replace('\n',''))

In [ ]:
df.reset_index(drop = True,inplace = True)

In [ ]:

# df['MPP1'] = df['MPP1'].apply(lambda x:x*10)
# df['MPP2'] = df['MPP2'].apply(lambda x:x*10)
# df['ML1'] = df['ML1'].apply(lambda x:x*10)
# df['ML2'] = df['ML2'].apply(lambda x:x*10)

In [ ]:
# post1 = df['post1']
# post2 = df['post2']
# # ml_label = df['ML_label']
# # mpp_label = df['MPP_label']
# mpp_label_1 = df['MPP1']
# mpp_label_2 = df['MPP2']

In [ ]:
df = df.dropna(axis = 0)

In [ ]:
# all_post = pd.concat([post1,post2],axis=0)
# all_post.reset_index(drop = True,inplace = True)
# mpp_all_value =  pd.concat([mpp_label_1,mpp_label_2],axis=0)
# mpp_all_value.reset_index(drop = True,inplace = True)

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
PRETRAINED_MODEL_NAME = "yiyanghkust/finbert-tone" 
# 获取预测模型所使用的tokenizer
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [ ]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn import linear_model
import matplotlib.pyplot as plt


In [ ]:
train = df[0:160]
validation = df[160:180]
test = df[176:200]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

In [ ]:
train_mpp_score = train['MPP1']
train_mpp_score2 = train['MPP2']
train_ml_score = train['ML1']
train_ml_score2 = train['ML2']
train_all_post = pd.concat([train['post1_en'],train['post2_en']],axis = 0)
train_all_mpp = pd.concat([train_mpp_score,train_mpp_score2],axis = 0)
train_all_ml = pd.concat([train_ml_score,train_ml_score2],axis = 0)

In [ ]:
test_label = test['MPP_label']
test_label = test['ML_label']

In [ ]:
x_test = test['post1_en']
x_test2 = test['post2_en']

In [ ]:

full_text_x = []
for i in train_all_post:
  i = tokenizer.tokenize(i)
  full_text_x.append(tokenizer.convert_tokens_to_ids(i))
all_pad_tokens = pad_sequences(full_text_x, maxlen=512, padding='post', truncating='post')



In [ ]:
type(all_pad_tokens)

numpy.ndarray

In [ ]:
regressor = KNeighborsRegressor()

#training the model
regressor.fit(all_pad_tokens, train_all_ml)

KNeighborsRegressor()

In [ ]:

full_text_t1 = []
full_text_t2 = []
for i in x_test:
  i = tokenizer.tokenize(i)
  full_text_t1.append(tokenizer.convert_tokens_to_ids(i))
for i in x_test2:
  i = tokenizer.tokenize(i)
  full_text_t2.append(tokenizer.convert_tokens_to_ids(i))
all_pad_tokens1 = pad_sequences(full_text_t1, maxlen=512, padding='post', truncating='post')
all_pad_tokens2 = pad_sequences(full_text_t2, maxlen=512, padding='post', truncating='post')

In [ ]:
y_pred1 = regressor.predict(all_pad_tokens1)
y_pred2 = regressor.predict(all_pad_tokens1)

In [ ]:
outcome = []
for i,j in zip(y_pred1,y_pred2):
  if i<j:
    outcome.append(1)
  else:
    outcome.append(0)


In [ ]:
accuracy_score(test_label,outcome)

0.4

In [ ]:
test_label

In [ ]:
for index,(true,pred) in enumerate(zip(test_label,outcome)):
  if true!=pred:
    print(index)

In [ ]:

def generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2):
  
  
  first_all_value = pd.concat([mpp_value1,ml_value1],axis=1)
  second_all_value = pd.concat([mpp_value2,ml_value2],axis=1)
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'mpp_label':torch.tensor(mpp_label.values,dtype = torch.long),
      'ml_label':torch.tensor(ml_label.values,dtype = torch.long),
      'first_all_value': torch.tensor(first_all_value.values, dtype=torch.float),
      'second_all_value': torch.tensor(second_all_value.values, dtype=torch.float),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

    

In [ ]:
class BertNerClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained('yiyanghkust/finbert-tone')
    self.drop = nn.Dropout(p=0.1)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask,token_type_ids,max_len,length1,length2):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids = token_type_ids

    )
    
    output = self.drop(output[0])
    
    cls_info = output[0][0].view(1,768)
    sent1_info = torch.mean(output[0][1:(length1+1)],dim=0).view(1,768)
    sent2_info = torch.mean(output[0][max_len:(max_len+length2)],dim=0).view(1,768)
    
    output = torch.cat((cls_info,sent1_info,sent2_info),dim = 0)
    
    
    
    return self.out(output)

In [ ]:
 
class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']
      self.mpp_label = data['mpp_label']
      self.ml_label = data['ml_label']
      self.first_all_value = data['first_all_value']
      self.second_all_value = data['second_all_value']
      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.mpp_label)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.mpp_label[index],self.ml_label[index],self.first_all_value[index],self.second_all_value[index],self.first_sen_len[index],self.second_sen_len[index]

In [ ]:
# train_post = all_post[:300]
# validation_post = all_post[300:400]
# # test_post1 = post1[180:200]

# train_mpp_value = mpp_all_value[:300]
# validation_mpp_value = mpp_all_value[300:400]
# # test_post2 = post2[180:200]

# # train_mpp_label = mpp_label[:160]
# # validation_mpp_label = mpp_label[160:180]
# # test_mpp_label = mpp_label[180:200]

# # train_ml_label = ml_label[:160]
# # validation_ml_label = ml_label[160:180]
# # test_ml_label = ml_label[180:200]
# # train_mpp_label = mpp_label[:160]

train = df[0:160]
validation = df[160:180]
test = df[180:200]


In [ ]:
train_post1 = train['post1_en']
train_post2 = train['post2_en']
train_mpp_label = train['MPP_label']
train_ml_label = train['ML_label']
train_mpp_value1 = train['MPP1'] 
train_mpp_value2 = train['MPP2']
train_ml_value1 = train['ML1']
train_ml_value2 = train['ML2']

In [ ]:
validation_post1 = validation['post1_en']
validation_post2 = validation['post2_en']
validation_mpp_label = validation['MPP_label']
validation_ml_label = validation['ML_label']
validation_mpp_value1 = validation['MPP1'] 
validation_mpp_value2 = validation['MPP2']
validation_ml_value1 = validation['ML1']
validation_ml_value2 = validation['ML2']

In [ ]:
test_post1 = test['post1_en']
test_post2 = test['post2_en']
test_mpp_label = test['MPP_label']
test_ml_label = test['ML_label']
test_mpp_value1 = test['MPP1'] 
test_mpp_value2 = test['MPP2']
test_ml_value1 = test['ML1']
test_ml_value2 = test['ML2']

In [ ]:
len(test)

16

In [ ]:
def generate_dataloader(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2,batch_size):
  data = generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader


In [ ]:
max_len = 256
batch_size = 1
train_dataloader = generate_dataloader(train_post1,train_post2,train_mpp_label,train_ml_label,max_len,train_mpp_value1,train_ml_value1,train_mpp_value2,train_ml_value2,batch_size)
# test_dataloader = generate_dataloader(test_post1,test_post2,test_mpp_label,test_ml_label,max_len,batch_size)
validation_dataloader = generate_dataloader(validation_post1,validation_post2,validation_mpp_label,validation_ml_label,max_len,validation_mpp_value1,validation_ml_value1,validation_mpp_value2,validation_ml_value2,batch_size)

In [ ]:
test_dataloader = generate_dataloader(test_post1,test_post2,test_mpp_label,test_ml_label,max_len,test_mpp_value1,test_ml_value1,test_mpp_value2,test_ml_value2,batch_size)

In [ ]:
EPOCHS = 100

# total_steps = len(train_data_loader) * EPOCHS

# scheduler = get_linear_schedule_with_warmup(
#   optimizer,
#   num_warmup_steps=0,
#   num_training_steps=total_steps
# )
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_fn = nn.MSELoss().to(device)


label_num = 2
model= BertNerClassifier(label_num).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

def train_and_eval(model, 
  train_loader, 
  validation_loader,
  loss_fn, 
  optimizer,  
  device,
  epoch,
  label_num 
  ):
  model.train()
  max_len = 256
  mpp_accuracy_state = 0
  ml_accuracy_state = 0
  for epo in range(epoch):
    
    train_loss = []
    
    global train_mpp_predictions 
    train_mpp_predictions = []
    global train_ml_predictions
    train_ml_predictions = []
    

    validation_loss = []
    
    validation_mpp_predictions = []
    validation_ml_predictions = []


    global real_train_mpp 
    real_train_mpp = []
    global real_train_ml
    real_train_ml = []
    real_validation_mpp = []
    real_validation_ml = []
    
   
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      token_type_ids = data[2].to(device)
      
      mpp_targets = data[3].to(device)
      ml_targets = data[4].to(device)
      first_targets = data[5].to(device)
      second_targets = data[6].to(device)
      first_sen_len = data[7].to(device)
      second_sen_len = data[8].to(device)
      
      

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
      )
      loss1 = loss_fn(outputs[0],torch.tensor([mpp_targets,ml_targets],dtype = torch.float).to(device))
      
      loss2 = loss_fn(outputs[1],first_targets.view(-1))
      
      loss3 = loss_fn(outputs[2],second_targets.view(-1))
      
      loss = loss1+loss2+loss3
 
      
      

      train_loss.append(loss.item())
      
      
      if outputs[0][0]>0.5:
        train_mpp_predictions.append(1)
      else:
        train_mpp_predictions.append(0)
      if outputs[0][1]>0.5:
        train_ml_predictions.append(1)
      else:
        train_ml_predictions.append(0)

      real_train_mpp.append(mpp_targets.cpu().item())
      real_train_ml.append(ml_targets.cpu().item())
      

         
      
    
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model.eval()

    with torch.no_grad():
      for data in validation_loader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_ids = data[2].to(device)
      
        mpp_targets = data[3].to(device)
        ml_targets = data[4].to(device)
        first_targets = data[5].to(device)
        second_targets = data[6].to(device)
        first_sen_len = data[7].to(device)
        second_sen_len = data[8].to(device)
      

        outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask,
          token_type_ids = token_type_ids,
          max_len = max_len,
          length1 = first_sen_len,
          length2 = second_sen_len
   
        )
        loss1 = loss_fn(outputs[0],torch.tensor([mpp_targets,ml_targets],dtype = torch.float).to(device))
        
        loss2 = loss_fn(outputs[1],first_targets.view(-1))
        
        loss3 = loss_fn(outputs[2],second_targets.view(-1))
        
        loss = loss1+loss2+loss3
 
      
      

        validation_loss.append(loss.item())
      
        if outputs[0][0]>0.5:
          validation_mpp_predictions.append(1)
        else:
          validation_mpp_predictions.append(0)
        if outputs[0][1]>0.5:
          validation_ml_predictions.append(1)
        else:
          validation_ml_predictions.append(0)
        # validation_ml_predictions += torch.sum(preds == ml_targets)

        real_validation_mpp.append(mpp_targets.cpu().item())
        real_validation_ml.append(ml_targets.cpu().item())
    
    train_mpp_accuracy = accuracy_score(real_train_mpp, train_mpp_predictions)
    train_ml_accuracy = accuracy_score(real_train_ml, train_ml_predictions)
    validation_mpp_accuracy = accuracy_score(real_validation_mpp, validation_mpp_predictions)
    validation_ml_accuracy = accuracy_score(real_validation_ml, validation_ml_predictions)
    
    final_train_loss = np.mean(train_loss)
 
    final_validation_loss = np.mean(validation_loss)
   

    if(validation_mpp_accuracy > mpp_accuracy_state):
      mpp_accuracy_state = validation_mpp_accuracy
      torch.save(model.state_dict(), 'fincom_mpp_bert_state.bin')
    if(validation_ml_accuracy > ml_accuracy_state):
      ml_accuracy_state = validation_ml_accuracy
      torch.save(model.state_dict(), 'fincom_ml_bert_state.bin')

    print(
'Epoch [{}] Train Totol Loss: {:.4f}, Train MPP Accuracy: {:.4f}, Train ML Accuracy: {:.4f}, Valid Total Loss: {:.4f}, Valid MPP Accuracy: {:.4f}, Valid ML Accuracy: {:.4f}'
    .format(epo+1,final_train_loss, train_mpp_accuracy, train_ml_accuracy,final_validation_loss,validation_mpp_accuracy,validation_ml_accuracy))

In [ ]:
train_and_eval(model, 
  train_dataloader, 
  validation_dataloader,
  loss_fn, 
  optimizer,  
  device,
  EPOCHS,
  label_num
  )

Epoch [1] Train Totol Loss: 0.6690, Train MPP Accuracy: 0.5125, Train ML Accuracy: 0.5062, Valid Total Loss: 0.3243, Valid MPP Accuracy: 0.4500, Valid ML Accuracy: 0.4000
Epoch [2] Train Totol Loss: 0.2544, Train MPP Accuracy: 0.8000, Train ML Accuracy: 0.7625, Valid Total Loss: 0.2903, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.4500
Epoch [3] Train Totol Loss: 0.1560, Train MPP Accuracy: 0.9625, Train ML Accuracy: 0.8750, Valid Total Loss: 0.3864, Valid MPP Accuracy: 0.6000, Valid ML Accuracy: 0.5000
Epoch [4] Train Totol Loss: 0.2066, Train MPP Accuracy: 0.9375, Train ML Accuracy: 0.8938, Valid Total Loss: 0.4693, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.5500
Epoch [5] Train Totol Loss: 0.1706, Train MPP Accuracy: 0.9250, Train ML Accuracy: 0.9437, Valid Total Loss: 0.5226, Valid MPP Accuracy: 0.3500, Valid ML Accuracy: 0.5500
Epoch [6] Train Totol Loss: 0.1131, Train MPP Accuracy: 0.9250, Train ML Accuracy: 0.9750, Valid Total Loss: 0.2961, Valid MPP Accuracy: 0.4500, 

In [ ]:
def test_prediction(model, 
  test_dataloader, 
  loss_fn, 
  optimizer,
  device
  ):
  
  test_mpp_predictions = []
  test_ml_predictions = []
  real_test_mpp = []
  real_test_ml= []

  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_ids = data[2].to(device)
      
        mpp_targets = data[3].to(device)
        ml_targets = data[4].to(device)
        first_targets = data[5].to(device)
        second_targets = data[6].to(device)
        first_sen_len = data[7].to(device)
        second_sen_len = data[8].to(device)
      

        outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask,
          token_type_ids = token_type_ids,
          max_len = max_len,
          length1 = first_sen_len,
          length2 = second_sen_len
   
        )
        # loss1 = loss_fn(outputs[0],torch.tensor([mpp_targets,ml_targets],dtype = torch.float).to(device))
        
        # loss2 = loss_fn(outputs[1],first_targets.view(-1))
        
        # loss3 = loss_fn(outputs[2],second_targets.view(-1))
        
        # loss = loss1+loss2+loss3
 
      
      

        
      
        if outputs[0][0]>0.5:
          test_mpp_predictions.append(1)
        else:
          test_mpp_predictions.append(0)
        if outputs[0][1]>0.5:
          test_ml_predictions.append(1)
        else:
          test_ml_predictions.append(0)
        # validation_ml_predictions += torch.sum(preds == ml_targets)

        real_test_mpp.append(mpp_targets.cpu().item())
        real_test_ml.append(ml_targets.cpu().item())

        test_mpp_accuracy = accuracy_score(real_test_mpp, test_mpp_predictions)
        test_ml_accuracy = accuracy_score(real_test_ml, test_ml_predictions)

  

  print('Test MMP ACCURACY_SCORE: {:.4f},Test ML ACCURACY_SCORE: {:.4f}'.format(test_mpp_accuracy,test_ml_accuracy))

In [ ]:
test_prediction(model, 
  test_dataloader, 
  loss_fn, 
  optimizer,
  device,
  
  )

Test MMP ACCURACY_SCORE: 0.6250,Test ML ACCURACY_SCORE: 0.4375


In [ ]:
model.load_state_dict(torch.load('fincom_ml_bert_state.bin', map_location=torch.device("cuda:0")))

<All keys matched successfully>

In [ ]:
Test MMP ACCURACY_SCORE: 0.6875,Test ML ACCURACY_SCORE: 0.6250

In [ ]:
Test MMP ACCURACY_SCORE: 0.6250,Test ML ACCURACY_SCORE: 0.4375  